# GANomaly ISM anomaly chirp

/home/qxslab/ty/ganomaly1


##  
##  best AUC 0.871

       具有大致轮廓，图像非常模糊，细节不清晰；AUC结果不稳定，会突然出现一个很高的值。例如0.871，其他时刻都比较低，0.54
       archive 0022
       /home/qxslab/anaconda3/bin/python /home/qxslab/ty/ganomaly/train.py --dataset folder --dataroot /home/qxslab/ty/data --isize 256 --nc 3 --nz 1000 --niter 1000 --display
    
    
    
##  
##  nz=1500 模型不好训练,无法捕捉到大致轮廓

        best AUC 0.56
       /home/qxslab/anaconda3/bin/python /home/qxslab/ty/ganomaly/train.py --dataset folder --dataroot /home/qxslab/ty/data --isize 256 --nc 3 --nz 1500 --niter 1000 --display
    
    
    
##  
##  改变learning rate 从默认的0.0002改成0.0001

       /home/qxslab/anaconda3/bin/python /home/qxslab/ty/ganomaly/train.py --dataset folder --dataroot /home/qxslab/ty/data --isize 256 --nc 3 --nz 1500 --niter 1000 --display --lr 0.0001
    
       最好的auc 0.591
    
    
##  
##  输出roc曲线

    调整acc的threshold，lib/evaluate.py line 35,从0.2调整为0.1
    打印score确定合适的label，更加正确的方法是输出roc曲线
    
    
    
    
##  
##  进一步降低nz为500
    
       /home/qxslab/anaconda3/bin/python /home/qxslab/ty/ganomaly/train.py --dataset folder --dataroot /home/qxslab/ty/data --isize 256 --nc 3 --niter 1000 --display --nz 500
       训练了286次，可以获得清晰的生成图像，但是auc只有0.4
       训练过程中出现的最大auc 0.818，不确定是在哪里出现的
       result saved in archive 0023
    
<img src='ganomaly_ism/nz500-256epoch.png'> 

       Q:

       为什么生成图像比较清晰但是auc很低  
    
       明确什么时候会保存参数：auc最大时保存参数
       err_ 由l2_loss产生
    
  
  
    
    
    
##  
##  restore model try to restore best auc
    
        python train.py --dataset restore_auc_0p818 --dataroot /home/qxslab/ty/data --isize 256 --nc 3 --niter 1000 --display --nz 500 --resume output/ganomaly/folder/train/weights
        
        最大0.842
        
 <img src='ganomaly_ism/hight_auc_bad_image.PNG'>       
    
    Q:

    为什么auc很高但是生成图像很差
    
    
##  
##  batch size 从64调整为128
    
    
##  
##  使用gamma变换
    
    gamma 最大auc 0.913， 在0.85附近波动

  <img src='ganomaly_ism/gamma_res.png'>  
  虽然auc很大，但是生成的图像为一片漆黑
  
  为此增加w_bce,使得生成的图像更真实


##  
##  使用gauss_blur变换


## 问题分析
训练过程中loss一直在降低，但是auc升高的并不明显，甚至会下降。这是因为loss的作用使得生成的图片越来越真实，导致出现异常的可能性也提高了，正常和异常图片的得分越来越一致。

核心问题是，图片越来越真实的过程与异常并不是完全矛盾的

应该使概率尽可能地分布在正常图片周围
ebgan

使用对抗自编码器，使得正常图片分布服从某一分布，即似然概率足够高，而异常图片似然概率足够低

ganomaly的目标函数与auc不直接相关，只是会在优化的过程中偶尔导致auc较大，目的性不强

异常信号是从没有出现过的，或者出现的次数很低

因此模型在竭尽全力地增强正常信号的概率的过程中，会使得未出现区域的概率降低

当前面临的问题是由于样本数量接近于无穷，所以每个样本的概率都近似为0
    
##  
##  /gamma/constrast/wavelet/EMBOSS
    
    
    
    重点试一下gamma/gamma_smooth/emboss GaussiBlur/GaussiBlur
    
    可能的原因是：背景中有很多噪点，到时gan无法学习到清晰的特征。因为如果gan学到了清晰的特征，虽然能够在有信号存在的地方降低loss，但是在更大的不存在信号的背景中loss很高。
    
    使用gamma变换抹除微弱信号，仅保留主要信号

                       <img src='ganomaly_ism/org.jpg'>
<img src='ganomaly_ism/org.jpg'>



                        <img src='ganomaly_ism/gamma.jpg'>
<img src='ganomaly_ism/gamma.jpg'>



                        <img src='ganomaly_ism/EMBOSS.jpg'>
<img src='ganomaly_ism/EMBOSS.jpg'>


                        <img src='ganomaly_ism/GaussianBlur.jpg'>
<img src='ganomaly_ism/GaussianBlur.jpg'>


                        <img src='ganomaly_ism/Gaussian_EMBOSS.jpg'>
<img src='ganomaly_ism/Gaussian_EMBOSS.jpg'>


    

    

    